In [1]:
from ReadData import ReadFile

In [2]:
samchild2014=ReadFile(2014,'samchild')
samchild2014.head()

Reading in file samchild for year 2014
Results:  Dataframe created with 13380 rows and 1245 columms.


,RECTYPE,SRVY_YR,HHX,INTV_QRT,INTV_MON,FMX,FPX,WTIA_SC,WTFA_SC,REGION,...,RSCL5_H5,RSCL6,CSHFLU12,CSHFLUNM,CSHFLUM1,CSHFLUY1,CSHSPFL1,CSHFLUM2,CSHFLUY2,CSHSPFL2
0,40,2014,13,1,1,1,3,49029,6039,4,...,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,2014,20,1,2,1,4,21905,3145,4,...,NaN,NaN,1,1.0,9.0,2013.0,2.0,NaN,NaN,NaN
2,40,2014,25,1,1,1,4,38460,5497,2,...,2.0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,2014,29,1,1,1,5,3742,592,2,...,0.0,0.0,1,1.0,10.0,2013.0,1.0,NaN,NaN,NaN
4,40,2014,34,1,2,1,2,49141,6553,1,...,2.0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
samadult2014=ReadFile(2014,'samadult')
samadult2014.head()

Reading in file samadult for year 2014
Results:  Dataframe created with 36697 rows and 3213 columms.


,RECTYPE,SRVY_YR,HHX,INTV_QRT,INTV_MON,FMX,FPX,WTIA_SA,WTFA_SA,REGION,...,ASIMUCH,ASIHIVT,ASIHIVWN,AWEBUSE,AWEBOFNO,AWEBOFTP,AWEBORP,AWEBEML,AWEBMNO,AWEBMTP
0,30,2014,7,1,1,1,2,233133,32518,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30,2014,14,1,2,1,1,33000,4105,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30,2014,16,1,3,1,1,34820,3795,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,2014,18,1,2,1,3,99903,16174,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,2014,19,1,2,1,3,60465,8750,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
familyxx2014=ReadFile(2014,'familyxx')
familyxx2014.head()

Reading in file familyxx for year 2014
Results:  Dataframe created with 45597 rows and 734 columms.


,RECTYPE,SRVY_YR,HHX,FMX,WTFA_FAM,FINT_Y_P,FINT_M_P,TELN_FLG,CURWRKN,TELCELN,...,HOUSEOWN,FGAH,FSSAPLYN,FSSAPLCT,FSDAPLYN,FSDAPLCT,FSNAP,FSNAPMYR,FWICYN,FWICCT
0,60,2014,4,1,5024,2014,1,1,2.0,1,...,2,2.0,2,0,1,1,1,6.0,2.0,0.0
1,60,2014,7,1,5296,2014,1,1,1.0,1,...,2,2.0,1,1,1,1,2,NaN,NaN,NaN
2,60,2014,13,1,6048,2014,1,1,2.0,1,...,2,2.0,2,0,2,0,2,NaN,2.0,0.0
3,60,2014,14,1,4201,2014,2,1,1.0,1,...,1,NaN,2,0,2,0,2,NaN,NaN,NaN
4,60,2014,16,1,4168,2014,3,1,1.0,1,...,1,NaN,2,0,2,0,2,NaN,NaN,NaN
